In [1]:
import pandas as pd
import numpy as np
import myfitnesspal
import datetime
from tqdm import tqdm

ModuleNotFoundError: No module named 'myfitnesspal'

In [ ]:
client = myfitnesspal.Client()

In [ ]:
start_date = datetime.date(2024, 4, 3)
end_date = datetime.date(2025, 11, 30)

columns = ['Date', 'Size', 'Food_Name', 'Calories', 'Carbohydrates', 'Fat', 'Protein', 'Sodium', 'Sugar']

df = pd.DataFrame(columns=columns)

In [ ]:
def pull_data_for_date(year, month, day):
    try:
        day_data = client.get_date(year, month, day)  
        
        meals = day_data.meals
        
        for meal in meals:
            if any(meal_type in meal._name.lower() for meal_type in ['dinner', 'breakfast', 'lunch', 'snacks']):
                for entry in meal._entries:
                    food_data = entry.__dict__ 
                    nutrition = food_data.get('_nutrition', {})  
                    
                    date = f"{day:02d}/{month:02d}/{str(year)[2:]}"
                    calories = nutrition.get('calories', np.nan)
                    carbohydrates = nutrition.get('carbohydrates', np.nan)
                    fat = nutrition.get('fat', np.nan)
                    protein = nutrition.get('protein', np.nan)
                    sodium = nutrition.get('sodium', np.nan)
                    sugar = nutrition.get('sugar', np.nan)
                    
                    serving_size = f"{food_data.get('_quantity', 'unknown')} {food_data.get('_unit', 'unknown')}"
                    
                    food_name = food_data.get('_name', 'unknown').strip()

                    df.loc[len(df)] = [date, serving_size, food_name, calories, carbohydrates, fat, protein, sodium, sugar]
    except Exception as e:
        print(f"Error pulling data for {year}-{month:02d}-{day:02d}: {e}")


In [ ]:
date_range = pd.date_range(start=start_date, end=end_date) 

for current_date in tqdm(date_range, desc="Processing Dates", unit="date"):
    year = current_date.year
    month = current_date.month
    day = current_date.day
    
    pull_data_for_date(year, month, day)

Processing Dates: 100%|██████████| 607/607 [09:34<00:00,  1.06date/s]


In [ ]:
df.to_csv('../data/dietary habit.csv', index=False)